In [1]:
from experiment_utils.experiments_helper import load_dense_index_from_disk, load_sparse_index_from_disk

In [2]:
import pyterrier as pt

if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from pyterrier.measures import RR, nDCG, MAP

eval_metrics = [RR @ 10, nDCG @ 10, MAP @ 100]

In [4]:
from encoders.snowflake_arctic_embed_m import SnowFlakeQueryEncoder

package = "Snowflake/"
model_name = "snowflake-arctic-embed-xs"
q_encoder = SnowFlakeQueryEncoder(package + model_name)
model_directory = 'snowflake'
path_to_root = "../../"
pipeline_name = "BM25 >> " + model_name

In [ ]:
from experiment_utils.experiments_helper import get_test_dev_sets
from fast_forward.util.pyterrier import FFScore
from fast_forward_indexes_library_enhancements.pipeline_transformers import FFInterpolateNormalized, EncodeUTF

dataset_name = "irds:beir/dbpedia-entity"
dev_set_name = "irds:beir/dbpedia-entity/dev"
dataset_test_name = "irds:beir/dbpedia-entity/test"
pipeline_name = "BM25 >> " + model_name

test_topics, test_qrels, dev_topics, dev_qrels = get_test_dev_sets(dataset_test_name, dev_set_name)

# Spare index
retriever = load_sparse_index_from_disk(dataset_name, path_to_root, in_memory=True)

# Dense index
dense_index = load_dense_index_from_disk(dataset_name, q_encoder, model_name, path_to_root, model_directory,
                                         in_memory=False)

test_topics = test_topics.sample(frac=0.02, random_state=42)

ff_score = FFScore(dense_index)
# ff_int = FFInterpolate(alpha=alpha)
ff_int = FFInterpolateNormalized(alpha=0.3)

encode_utf = EncodeUTF()

pipeline1 = retriever % 100 >> ff_score >> ff_int
pipeline2 = retriever % 100 >> encode_utf >> ff_score >> ff_int

# pipeline1 = retriever % 100
# pipeline2 = retriever % 100 >> encode_utf

In [9]:
df1 = retriever(test_topics)
filtered_df = df1[df1['docno'].str.contains('Co_Stomp', case=False, na=False)]
import ast

ast.literal_eval(filtered_df['docno'].iloc[0]).decode("utf-8")

'Co_Stompé'

In [6]:
pipeline2(test_topics)

,qid,docno,query,score
0,QALD2_tr-24,<dbpedia:The_Ascent_of_Rum_Doodle>,which mountain is the highest after the annapurna,0.439051
1,QALD2_tr-24,<dbpedia:Kang_Guru>,which mountain is the highest after the annapurna,0.615751
2,QALD2_tr-24,<dbpedia:Annapurna_(book)>,which mountain is the highest after the annapurna,0.700364
3,QALD2_tr-24,<dbpedia:Tilicho_Peak>,which mountain is the highest after the annapurna,0.507932
4,QALD2_tr-24,<dbpedia:Ian_Clough>,which mountain is the highest after the annapurna,0.465899
...,...,...,...,...
795,INEX_XER-86,<dbpedia:List_of_countries_by_number_of_mobile...,list of countries in world war two,0.582014
796,INEX_XER-86,<dbpedia:List_of_countries_by_electricity_expo...,list of countries in world war two,0.554434
797,INEX_XER-86,<dbpedia:List_of_countries_by_electricity_impo...,list of countries in world war two,0.539364
798,INEX_XER-86,<dbpedia:List_of_Little_League_World_Series_ap...,list of countries in world war two,0.495120


In [11]:
df2 = (retriever % 100 >> encode_utf)(test_topics)
df2

,qid,docid,docno,rank,score,query
0,125440,4614283,b'Stomp_the_Yard_(soundtrack)',0,36.388773,stomp the yard has meagan good in a minor role
1,125440,4601990,b'Stomp_the_Yard',1,35.434146,stomp the yard has meagan good in a minor role
2,125440,1099148,b'Collins_Pennie',2,32.134281,stomp the yard has meagan good in a minor role
3,125440,4552761,b'Stomp_the_Yard:_Homecoming',3,29.729887,stomp the yard has meagan good in a minor role
4,125440,2153633,b'Harry_Shum_Jr.',4,26.950157,stomp the yard has meagan good in a minor role
...,...,...,...,...,...,...
95,125440,4489570,b'Spiers_and_Boden',95,17.940394,stomp the yard has meagan good in a minor role
96,125440,4599706,b'Sustain_(album)',96,17.940394,stomp the yard has meagan good in a minor role
97,125440,597033,b'Arthur_Whittington',97,17.923718,stomp the yard has meagan good in a minor role
98,125440,2609731,b'Kenny_Mims',98,17.902330,stomp the yard has meagan good in a minor role


In [16]:
row = df1[df1['docno'] == 'Co_Stomp']
print(row)

       qid    docid     docno  rank      score  \
16  125440  1171956  Co_Stomp    16  22.429014   

                                             query  
16  stomp the yard has meagan good in a minor role  


In [17]:
print('Co_Stomp'.encode("utf-8"))

b'Co_Stomp'


In [20]:
df2 = pipeline2(test_topics)
df2

,qid,docid,docno,rank,score,query
0,125440,4614283,b'Stomp_the_Yard_(soundtrack)',0,36.388773,stomp the yard has meagan good in a minor role
1,125440,4601990,b'Stomp_the_Yard',1,35.434146,stomp the yard has meagan good in a minor role
2,125440,1099148,b'Collins_Pennie',2,32.134281,stomp the yard has meagan good in a minor role
3,125440,4552761,b'Stomp_the_Yard:_Homecoming',3,29.729887,stomp the yard has meagan good in a minor role
4,125440,2153633,b'Harry_Shum_Jr.',4,26.950157,stomp the yard has meagan good in a minor role
...,...,...,...,...,...,...
95,125440,4489570,b'Spiers_and_Boden',95,17.940394,stomp the yard has meagan good in a minor role
96,125440,4599706,b'Sustain_(album)',96,17.940394,stomp the yard has meagan good in a minor role
97,125440,597033,b'Arthur_Whittington',97,17.923718,stomp the yard has meagan good in a minor role
98,125440,2609731,b'Kenny_Mims',98,17.902330,stomp the yard has meagan good in a minor role


In [21]:
row = df2[df2['docno'] == 'Co_Stomp'.encode("utf-8")]
print(row)

       qid    docid        docno  rank      score  \
16  125440  1171956  b'Co_Stomp'    16  22.429014   

                                             query  
16  stomp the yard has meagan good in a minor role  


In [22]:
pipeline3 = retriever % 100 >> encode_utf >> ff_score

In [23]:
df3 = pipeline3(test_topics)
df3

no vectors for Co_Stomp
no vectors for Hiroki_Got


,qid,docno,score_0,score,query


In [24]:
pipeline4 = retriever % 100 >> ff_score

In [26]:
!pip install pydev_jupyter_utils

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pydev_jupyter_utils (from versions: none)
ERROR: No matching distribution found for pydev_jupyter_utils


In [28]:
pipeline4 = retriever % 100 >> ff_score
% debug pipeline4(test_topics)

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()



no vectors for Co_Stomp
no vectors for Hiroki_Got


In [3]:
from general_dense_indexers.dense_index_one_dataset import get_dataset_name


def docs_iter(dataset):
    for d in dataset.get_corpus_iter():
        yield {'docno': str(d['docno'].encode('utf-8')), 'text': d['text']}


def index_one(prefix_dataset, dataset_name, max_doc_id_length):
    if not pt.started():
        pt.init(tqdm="notebook")

    dataset = pt.get_dataset(prefix_dataset + dataset_name)

    index_path = "./sparse_indexes/sparse_index_" + get_dataset_name(prefix_dataset + dataset_name)
    indexer = pt.IterDictIndexer(index_path, meta={'docno': max_doc_id_length})

    if 'dbpedia' in dataset_name or 'fever' in dataset_name:
        indexer.index(docs_iter(dataset), fields=["text"])
    else:
        indexer.index(dataset.get_corpus_iter(), fields=["text"])


prefix_dataset = "irds:beir/"
dataset_name = "dbpedia-entity"
max_id_length = 200
index_one(prefix_dataset, dataset_name, max_id_length)

[INFO] [starting] building docstore
[INFO] [starting] opening zip file                                              
[INFO] [finished] opening zip file s]                                        
docs_iter: 100%|██████████████████| 4635922/4635922 [01:19<00:00, 57955.83doc/s]
[INFO] [finished] docs_iter: [01:19] [4635922doc] [57955.25doc/s]
[INFO] [finished] building docstore [01:20]
beir/dbpedia-entity documents:   1%|          | 33689/4635922 [00:05<07:27, 10286.98it/s]

16:51:52.302 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (b'<dbpedia:List_of_biblical_names_starting_with_C>') - further warnings are suppressed


beir/dbpedia-entity documents: 100%|██████████| 4635922/4635922 [06:45<00:00, 11439.90it/s]


17:01:29.559 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 336 empty documents


In [6]:
print(str('Co_Stompé'.encode("utf-8")) == 'Co_Stompé'.encode("utf-8"))

AttributeError: 'str' object has no attribute 'hex'

In [9]:
import ast

x = str('Co_Stompé'.encode("utf-8"))

# Converting the string representation back to bytes
original_bytes = ast.literal_eval(x)

print(original_bytes == 'Co_Stompé'.encode("utf-8"))

True


In [5]:
from experiment_utils.experiments_helper import get_test_dev_sets, default_test_pipeline

dataset_name = "irds:beir/dbpedia-entity"
dev_set_name = "irds:beir/dbpedia-entity/dev"
dataset_test_name = "irds:beir/dbpedia-entity/test"
pipeline_name = "BM25 >> " + model_name

test_topics, test_qrels, dev_topics, dev_qrels = get_test_dev_sets(dataset_test_name, dev_set_name)

dev_topics = dev_topics.sample(frac= 0.1, random_state=42)
test_topics = test_topics.sample(frac= 0.1, random_state=42)
#
# print(dev_topics)
# print("ccccc")
# print(test_topics)
res, alpha = default_test_pipeline(dataset_name, test_topics, test_qrels, q_encoder,
                      eval_metrics, model_name, pipeline_name, path_to_root, model_directory,
                      dev_topics, dev_qrels, timed=True, in_memory_sparse=True,
                      in_memory_dense=False)
res

GridScan: 100%|██████████| 9/9 [00:21<00:00,  2.33s/it]


Best ndcg_cut.10 is 0.049423
Best setting is ['<fast_forward_indexes_library_enhancements.pipeline_transformers.FFInterpolateNormalized object at 0x7f0cd0159390> alpha=0.1']
Experiment took 127.699 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,dbpedia_entity: BM25 >> snowflake-arctic-embed-xs,0.685863,0.361272,0.241665
